In [1]:
import cv2
import kornia as K
import kornia.feature as KF
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch

import kornia_moons
from kornia_moons.feature import *
from PIL import Image


In [2]:
doe_name = 'L1L2L3R_1010'

im_path =  '/data/s0/kyusu/NeurIPS_rebuttal_PCK/' + doe_name + '/'
udc_dataset = pd.read_csv("./udc.csv")
udc_dataset.species.value_counts().head(8)

udc    4680
Name: species, dtype: int64

In [3]:
udc_list = udc_dataset["individual_id"].values.tolist()
udc_list = list(set(udc_list))

print(len(udc_list))

2340


In [4]:
def load_torch_image(fname):
    img = K.image_to_tensor(cv2.imread(fname), False).float() /255.
    img = K.color.bgr_to_rgb(img)
    return img


In [5]:
global MAX_SIZE 
MAX_SIZE = 1792

DRAW = False

In [6]:
torch.cuda.empty_cache()

In [7]:
import torch
import torch.cuda
import torchvision
from PIL import Image
import torchvision.transforms as transforms

def load_torch_image(fname, device):
    img = Image.open(fname).convert("RGB")
    img = transforms.Resize((1280, 1792))(img) # Case 1: Resize to half in order to follow Feng's max(H,W), which is 1024.
                                               # Case 2: If we set MAX_SIZE = 1792 with image size (1280,1792), then the alignment accuracy increases.
                                               # Case 3: If we set MAX_SIZE = 3584 with image size (2560,3584), then the alignment accuracy would be consistent.
                                               # To reduce GPU memory usage, we opt for Case 2 and, accordingly, utilize our image's max(H, W).
    img = transforms.ToTensor()(img)
    img = img.unsqueeze(0).to(device)
    return img


def match_and_draw_gpu(im_path, img_in1, img_in2):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    img1 = load_torch_image(im_path + img_in1, device)
    img2 = load_torch_image(im_path + img_in2, device)
    
    matcher = KF.LoFTR(pretrained='outdoor')
    matcher = torch.nn.DataParallel(matcher)
    matcher = matcher.to(device)

    input_dict = {
        "image0": K.color.rgb_to_grayscale(img1).to(device), # Convert to grayscale and send to GPU
        "image1": K.color.rgb_to_grayscale(img2).to(device)  # Convert to grayscale and send to GPU
    }

    with torch.no_grad():
        correspondences = matcher(input_dict)

    if DRAW:
        mkpts0 = correspondences['keypoints0'].cpu().numpy()
        mkpts1 = correspondences['keypoints1'].cpu().numpy()

        H, inliers = cv2.findFundamentalMat(mkpts0, mkpts1, cv2.USAC_MAGSAC, 0.5, 0.999, 100000)
        inliers = inliers > 0

        kornia_moons.feature.draw_LAF_matches(
            KF.laf_from_center_scale_ori(torch.from_numpy(mkpts0).view(1, -1, 2),
                                        torch.ones(mkpts0.shape[0]).view(1, -1, 1, 1),
                                        torch.ones(mkpts0.shape[0]).view(1, -1, 1)), 

            KF.laf_from_center_scale_ori(torch.from_numpy(mkpts1).view(1, -1, 2),
                                        torch.ones(mkpts1.shape[0]).view(1, -1, 1, 1),
                                        torch.ones(mkpts1.shape[0]).view(1, -1, 1)), 

            torch.arange(mkpts0.shape[0]).view(-1, 1).repeat(1, 2),
            K.tensor_to_image(img1),
            K.tensor_to_image(img2),
            inliers,
            draw_dict={'inlier_color': (0.2, 1, 0.2),
                    'tentative_color': None,
                    'feature_color': (0.2, 0.5, 1), 'vertical': False}
        )
    
    return correspondences


In [8]:
def calculate_pck(correspondences, ind, pck_0p002_arr, pck_0p005_arr, pck_0p01_arr, pck_0p03_arr, pck_0p10_arr, dirname):
    # Keypoint coordinates for last prediction - only for showing structure 
    key_gt = correspondences['keypoints0'].cpu().numpy().T
    key_in = correspondences['keypoints1'].cpu().numpy().T

    ind = int(ind)
    np.savetxt('./'+dirname+'/keypoints_'+str(ind)+'_gt.txt', key_gt, delimiter=',', fmt='%.1f')
    np.savetxt('./'+dirname+'/keypoints_'+str(ind)+'_input.txt', key_in, delimiter=',', fmt='%.6f')

    diff = np.abs(key_gt - key_in)
    diff = np.ndarray.flatten(diff)
    count_0p002, count_0p005, count_0p01, count_0p03, count_0p10 = 0, 0, 0, 0, 0

    for i in range(len(diff)):
        if diff[i] <= MAX_SIZE * 0.002:
            count_0p002 += 1
        if diff[i] <= MAX_SIZE * 0.005:
            count_0p005 += 1    
        if diff[i] <= MAX_SIZE * 0.01:
            count_0p01 += 1
        if diff[i] <= MAX_SIZE * 0.03:
            count_0p03 += 1
        if diff[i] <= MAX_SIZE * 0.10:
            count_0p10 += 1

    pck_0p002, pck_0p005, pck_0p01, pck_0p03, pck_0p10 = count_0p002 / len(diff), count_0p005 / len(diff), count_0p01 / len(diff), count_0p03 / len(diff), count_0p10 / len(diff)
    print(".......Each PCK:", pck_0p002, pck_0p005, pck_0p01, pck_0p03, pck_0p10)

    pck_0p002_arr.append(pck_0p002), pck_0p005_arr.append(pck_0p005), pck_0p01_arr.append(pck_0p01), pck_0p03_arr.append(pck_0p03), pck_0p10_arr.append(pck_0p10)
    print(".......AVG PCK:", np.mean(pck_0p002_arr), np.mean(pck_0p005_arr), np.mean(pck_0p01_arr), np.mean(pck_0p03_arr), np.mean(pck_0p10_arr))

    np.savetxt('./'+dirname+'/pck_0p002.txt', pck_0p002_arr, delimiter=',', fmt='%.4f')
    np.savetxt('./'+dirname+'/pck_0p005.txt', pck_0p005_arr, delimiter=',', fmt='%.4f')
    np.savetxt('./'+dirname+'/pck_0p01.txt', pck_0p01_arr, delimiter=',', fmt='%.4f')
    np.savetxt('./'+dirname+'/pck_0p03.txt', pck_0p03_arr, delimiter=',', fmt='%.4f')
    np.savetxt('./'+dirname+'/pck_0p10.txt', pck_0p10_arr, delimiter=',', fmt='%.4f')
    np.savetxt('./'+dirname+'/pck_avg_all.txt', [pck_0p002_arr, pck_0p005_arr, pck_0p01_arr, pck_0p03_arr, pck_0p10_arr], delimiter=',', fmt='%.4f')

    return pck_0p002_arr, pck_0p005_arr, pck_0p01_arr, pck_0p03_arr, pck_0p10_arr

In [10]:
import os

dirname = doe_name
if not os.path.exists(dirname):
    os.makedirs(dirname)

In [11]:
pck_0p002_arr, pck_0p005_arr, pck_0p01_arr, pck_0p03_arr, pck_0p10_arr = [], [], [], [], []

for i in udc_list:
    torch.cuda.empty_cache()
    query_str = "individual_id == " + str(i)
    img_to_draw = [file for file in udc_dataset.query(query_str).image]

    udc_1 = img_to_draw[1]
    udc_2 = img_to_draw[0]
    print(f'Matching: {udc_1} to {udc_2}')

    correspondences = match_and_draw_gpu(im_path, udc_2, udc_1)

    pck_0p002_arr, pck_0p005_arr, pck_0p01_arr, pck_0p03_arr, pck_0p10_arr = calculate_pck(correspondences, i, pck_0p002_arr, pck_0p005_arr, pck_0p01_arr, pck_0p03_arr, pck_0p10_arr, dirname)

Matching: 1_input.png to 1_GT.png
.......Each PCK: 0.9664550569903333 0.9878083970567018 0.9906218438897706 0.9950945029577262 0.9997114413504545
.......AVG PCK: 0.9664550569903333 0.9878083970567018 0.9906218438897706 0.9950945029577262 0.9997114413504545
Matching: 2_input.png to 2_GT.png
.......Each PCK: 0.974399539767007 0.992952682295412 0.9964044297425572 0.9971954551991946 0.9995685315691069
.......AVG PCK: 0.9704272983786701 0.9903805396760569 0.9935131368161638 0.9961449790784604 0.9996399864597807
Matching: 3_input.png to 3_GT.png
.......Each PCK: 0.9712217084448771 0.9933682161109759 0.9956193721100024 0.9964711608663909 0.9978096860550012
.......AVG PCK: 0.9706921017340724 0.9913764318210299 0.9942152152474434 0.996253706341104 0.9990298863248542
Matching: 4_input.png to 4_GT.png
.......Each PCK: 0.8983280614550384 0.9881759301099563 0.9947281217050761 0.9960084350052719 0.9981171863232414
.......AVG PCK: 0.952601091664314 0.9905763063932614 0.9943434418618515 0.996192388507

In [12]:
torch.cuda.empty_cache()